In [254]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Run this to attack a trained model via TrainModel. 
Use the "loadFullModel" submethod to load in an already trained model (trained via TrainModel)
The main attack function is "runAttacks" which runs attacks on trained models
"""
import warnings
warnings.filterwarnings("ignore")

import os
import tensorflow as tf
#tf.compat.v1.enable_eager_execution
os.environ['CUDA_VISIBLE_DEVICES'] = ''
from cleverhans.attacks import Noise, CarliniWagnerL2, MaxConfidence, FastGradientMethod, BasicIterativeMethod, DeepFool, MomentumIterativeMethod, ProjectedGradientDescent
from Model_Implementations import Model_Softmax_Baseline, Model_Logistic_Baseline, Model_Logistic_Ensemble, Model_Tanh_Ensemble, Model_Tanh_Baseline
from tensorflow.keras.datasets import mnist, cifar10
from tensorflow.keras import backend
import numpy as np
import scipy.linalg
from scipy import stats
import matplotlib.pyplot as plt

In [255]:
model_path = 'models/'  #path with saved model parameters 
sess =  backend.get_session()
backend.set_learning_phase(0) #need to do this to get CleverHans to work with batchnorm


#Dataset-specific parameters - should be same as those used in TrainModel

#CIFAR
# DATA_DESC = 'CIFAR10'; (X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
# epochs=None; weight_save_freq=None
# num_classes=10  #how many classes (categories) are in this dataset?
# Y_train = np.squeeze(Y_train); Y_test = np.squeeze(Y_test)
# num_filters_std = [32, 64, 128]; num_filters_ens=[32, 64, 128]; num_filters_ens_2=16; dropout_rate_std=0.0; dropout_rate_ens=0.0; weight_decay = 0 
# model_rep_baseline=2; model_rep_ens=2; DATA_AUGMENTATION_FLAG=1; BATCH_NORMALIZATION_FLAG=1
# num_channels = 3; inp_shape = (32,32,3); lr=1e-4; batch_size=80;
# noise_stddev = 0.032; blend_factor = .032

#MNIST
DATA_DESC = 'MNIST'; (X_train, Y_train), (X_test, Y_test) = mnist.load_data()
Y_train = np.squeeze(Y_train); Y_test = np.squeeze(Y_test)
num_channels = 1; inp_shape = (28,28,1); num_classes=10
#MODEL-SPECIFIC PARAMETERS: MNIST
#PARAMETERS RELATED TO SGD OPTIMIZATION
epochs=None; weight_save_freq=None; batch_size=80; lr=3e-4; 
#MODEL DEFINTION PARAMETERS
num_filters_std = [64, 64, 64]; num_filters_ens=[32, 32, 32]; num_filters_ens_2=4; 
dropout_rate_std=0.0; dropout_rate_ens=0.0; weight_decay = 0 
noise_stddev = 0.3; blend_factor=0.3; 
model_rep_baseline=1; model_rep_ens=2; 
DATA_AUGMENTATION_FLAG=0; BATCH_NORMALIZATION_FLAG=0

#Attack parameters
eps_val = 8/255.0; PGD_iters = 200; eps_iter=(2/3)*eps_val; 
eps_range = np.linspace(0, 0.33, 10)
noise_eps=0.1


# DATA PRE-PROCESSING
X_train = (X_train/255).astype(np.float32);  X_test = (X_test/255).astype(np.float32)
#reshape (add third (image) channel)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2],num_channels); X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2],num_channels)
X_valid = X_test[1000:2000]; Y_valid = Y_test[1000:2000]; #validation data, used to attack model
X_train = X_train-0.5; X_test = X_test-0.5; X_valid = X_valid-0.5; #map to range (-0.5,0.5)
data_dict = {'X_train':X_train, 'Y_train_cat':Y_train, 'X_test':X_test, 'Y_test_cat':Y_test}
X_random = np.random.rand(X_valid.shape[0],X_valid.shape[1],X_valid.shape[2],X_valid.shape[3])-0.5; X_random = X_random.astype(np.float32)


# def output_activation(x):
#     return tf.nn.softmax(x) 

#Model definition of the model we want to attack; should be same as the definition used in TrainModel
name = 'softmax_baseline'+'_'+DATA_DESC; num_chunks=1
M = np.eye(num_classes).astype(np.float32)
def output_activation(x):
    return tf.nn.softmax(x) 
base_model=None
params_dict = {'weight_decay':weight_decay, 'num_filters_std':num_filters_std, 'BATCH_NORMALIZATION_FLAG':BATCH_NORMALIZATION_FLAG, 'DATA_AUGMENTATION_FLAG':DATA_AUGMENTATION_FLAG, 'M':M, 'model_rep':model_rep_baseline, 'base_model':base_model, 'num_chunks':num_chunks, 'output_activation':output_activation,  'batch_size':batch_size, 'epochs':epochs, 'lr':lr, 'dropout_rate':dropout_rate_std,  'blend_factor':blend_factor, 'inp_shape':inp_shape, 'noise_stddev':noise_stddev, 'weight_save_freq':weight_save_freq, 'name':name, 'model_path':model_path}
m4 = Model_Softmax_Baseline(data_dict, params_dict)
m4.loadFullModel() #load in the saved model, which should have already been trained first via TrainModel

m4.legend = 'TEns16'; 
m4.X_valid = X_valid; m4.Y_valid = Y_valid; 
m4.X_test = X_test; m4.Y_test = Y_test; 
m4.X_random = X_random; 
m4.minval = -0.5; m4.maxval = 0.5



def benignAccuracy(model, X, Y):
    
    acc_vec=[]; probs_benign_list=[]
    for rep in np.arange(0, X.shape[0], 1000):
        x = X[rep:rep+1000]
        probs_benign = sess.run(model.predict(tf.convert_to_tensor(x))) 
        acc = np.mean(np.argmax(probs_benign, 1)==Y[rep:rep+1000])
        acc_vec += [acc]
        probs_benign_list += list(np.max(probs_benign, 1))

        
    acc = np.mean(acc_vec)        
    print("Accuracy for model " + model.params_dict['name'] + " : ", acc)    
    return probs_benign_list


def wbAttack(model, attack, att_params, X, Y):
    sess =  backend.get_session()
    modelCH = model.modelCH()
    adv_model = attack(modelCH, sess=sess) 
    
    acc_vec=[]; probs_adv_list=[]
    inc=250
    for rep in np.arange(0, X.shape[0], inc):
        x = X[rep:rep+inc]
        y = Y[rep:rep+inc]
        X_adv = adv_model.generate(tf.convert_to_tensor(x), **att_params).eval(session=sess)  
        print(X_adv.dtype)
        preds = np.argmax(sess.run(model.predict(tf.convert_to_tensor(X_adv))), 1)
        acc =  np.mean(np.equal(preds, y))
        probs_adv = np.max(sess.run(model.predict(tf.convert_to_tensor(X_adv))), 1)
        probs_adv = probs_adv[preds != y]
        acc= np.mean(np.equal(preds, y))
        acc_vec += [acc]
        probs_adv_list += list(probs_adv)

        
    acc = np.mean(acc_vec)        
    print("Adv accuracy for model " + model.params_dict['name'] + " : ", acc)    
    return probs_adv_list, acc, X_adv, y


import timeit

start = timeit.default_timer()

stop = timeit.default_timer()
print('Time: ', stop - start)  


def runAttacks(models_list):
    #CW attack
    for model in models_list:
        
        print(""); print(""); print("");
        print("Running tests on model: ", model.params_dict['name'])
        start = timeit.default_timer()
        print("Clean accuracy of model:")
        probs_benign = benignAccuracy(model, model.X_test, model.Y_test)
        print("")
        stop = timeit.default_timer()
        print('Benign Time: ', stop - start)  
        
        start = timeit.default_timer()
        print("Running PGD attack:")
        att_params = {'clip_min': model.minval, 'clip_max':model.maxval, 'eps':eps_val, 'eps_iter':eps_iter, 'nb_iter':PGD_iters,'ord':np.inf}
        probs_adv_pgd, junk, X_adv_pgd, y_adv_pgd = wbAttack(model, ProjectedGradientDescent, att_params, model.X_valid, model.Y_valid)
        print("")
        stop = timeit.default_timer()
        print('PGD Time: ', stop - start) 
        
#         start = timeit.default_timer()
#         print("Running CW attack:")
#         att_params = {'clip_min': model.minval, 'clip_max':model.maxval,  'binary_search_steps':10, 'learning_rate':1e-3}
#         probs_adv_cw, junk, X_adv, y = wbAttack(model, CarliniWagnerL2, att_params, model.X_valid[0:100], model.Y_valid[0:100])
#         print("")
#         stop = timeit.default_timer()
#         print('CW Time: ', stop - start) 
        
#         start = timeit.default_timer()
#         print("Running Blind Spot attack, alpha=0.8:")
#         att_params = {'clip_min': model.minval, 'clip_max':model.maxval,  'binary_search_steps':10, 'learning_rate':1e-3}
#         probs_adv_bsa, junk, X_adv, y = wbAttack(model, CarliniWagnerL2, att_params, 0.8*model.X_valid[0:100], model.Y_valid[0:100])
#         print("")
#         stop = timeit.default_timer()
#         print('BSA Time: ', stop - start) 
                
        #Random ATTACK (0 SNR inputs)
#         start = timeit.default_timer()
#         print("Running random attack:")
#         probs_random = np.max(sess.run(model.predict(tf.convert_to_tensor(model.X_random))), 1)
#         print('Prob. that ', model.params_dict['name'], ' < 0.9 on random data: ', np.mean(probs_random<0.9))
#         stop = timeit.default_timer()
#         print('Random Time: ', stop - start) 
        
#         #Noise ATTACK (low SNR inputs)
#         start = timeit.default_timer()
#         print("Running Noise attack:")
#         att_params = {'clip_min': model.minval, 'clip_max':model.maxval, 'eps':noise_eps}
#         probs_noise, junk, X_adv, y = wbAttack(model, Noise, att_params, model.X_valid, model.Y_valid)
#         print("")
#         stop = timeit.default_timer()
#         print('Running Noise Time: ', stop - start) 
        
#     return probs_benign, probs_adv_pgd, probs_adv_cw, probs_adv_bsa, probs_random, probs_noise, X_adv_pgd, y_adv_pgd
    return probs_adv_pgd, X_adv_pgd, y_adv_pgd #for plotting probs



models_list = [m4]
# probs_benign, probs_adv_pgd, probs_adv_cw, probs_adv_bsa, probs_random, probs_noise, X_adv_pgd, y_adv_pgd = runAttacks(models_list)
probs_adv_pgd, X_adv_pgd, y_adv_pgd = runAttacks(models_list) #for ploting probs


Model: "model_59"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 28, 28, 1)]       0         
_________________________________________________________________
gaussian_noise_3 (GaussianNo (None, 28, 28, 1)         0         
_________________________________________________________________
lambda_9 (Lambda)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 28, 28, 64)        1664      
_________________________________________________________________
lwta__conv2d__activation_18  ((None, 28, 28, 64), (Non 0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 14, 14, 64)        36928     
_________________________________________________________________
lwta__conv2d__activation_19  ((None, 14, 14, 64), (Non 0  

KeyboardInterrupt: 

In [ ]:
# DAA 4rd try according to tf2 cleverhans 

# WHY IS X_ADV NEGATIVE?

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from scipy.spatial.distance import pdist, squareform
from matplotlib import pyplot as plt

c = 1.1
import time

class LinfBLOBAttack:
    def __init__(self, model, epsilon, k, a, random_start):
        self.model = model
        self.epsilon = epsilon
        self.k = k
        self.a = a
        self.rand = random_start

        c = 1.1
        print('c: ', c)

    def perturb(self, x_nat, x_adv, y, sess): 
        batch_size = x_adv.shape[0]
        for epoch in range(10):
            print(epoch)
            x_adv = tf.convert_to_tensor(x_adv)
            x_adv = tf.dtypes.cast(x_adv, tf.float32)
            with tf.GradientTape(persistent=True) as g:
                g.watch(x_adv)
    #             out = tf.argmax(m4.model(x_nat),axis=1)
    #             out = tf.expand_dims(out, axis=1)
    #             out = tf.dtypes.cast(out, tf.float64)
    #             y = tf.dtypes.cast(y, tf.int64)
    #             loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=out)
                out = m4.model(x_adv)
                y = tf.expand_dims(y, axis=1)
                out = tf.dtypes.cast(out, tf.float32)
                y = tf.dtypes.cast(y, tf.int64)
                loss = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=out)

            grad = g.gradient(loss, x_adv)
            grad = grad.eval(session=sess)
            x_adv = x_adv.eval(session=sess)
            grad = np.reshape(grad, [grad.shape[0], grad.shape[1]*grad.shape[2]*grad.shape[3]])
            kxy, dxkxy = self.svgd_kernel(x_adv)
            x_adv = np.reshape(x_adv, [x_adv.shape[0], x_adv.shape[1]*x_adv.shape[2]*x_adv.shape[3]])
            x_adv += self.a * np.sign(c*(-(np.matmul(kxy, -grad) + dxkxy)/batch_size) + grad)
            
            #we need to put back the dimensions since our model has a 4-dimensional input
            x_adv = np.reshape(x_adv, [x_adv.shape[0], 28, 28, 1])
            
            x_adv = np.clip(x_adv, x_nat - self.epsilon, x_nat + self.epsilon) 
            
            x_adv = np.clip(x_adv, 0, 1) # ensure valid pixel range

        return x_adv
    
    def svgd_kernel(self, theta):
        theta = np.reshape(theta, [theta.shape[0], theta.shape[1]*theta.shape[2]])
        sq_dist = pdist(theta)
        pairwise_dists = squareform(sq_dist)**2

        h = np.median(pairwise_dists)  
        h = np.sqrt(0.5 * h / np.log(theta.shape[0]))

        # compute the rbf kernel
        Kxy = np.exp( -pairwise_dists / h**2 / 2)

        dxkxy = -np.matmul(Kxy, theta)
        sumkxy = np.sum(Kxy, axis=1)
        for i in range(theta.shape[1]):
            dxkxy[:, i] = dxkxy[:,i] + np.multiply(theta[:,i],sumkxy)
        dxkxy = dxkxy / (h**2)
        return (Kxy, dxkxy)


if __name__ == '__main__':
    import json
    import sys
    import math
    
    sess =  backend.get_session()

    attack = LinfBLOBAttack(m4,
                         epsilon=0.3,
                         k=200,
                         a=0.01,
                         random_start=True)

    num_eval_examples = 1000
    eval_batch_size = 200
    num_batches = int(math.ceil(num_eval_examples / eval_batch_size))

    print('Iterating over {} batches'.format(num_batches))

    #     x_adv_final = np.copy(mnist.test.images)

    for restart in range(50):
      # Initialize permutation
        permutation = np.arange(num_eval_examples)
        idx = np.arange(num_eval_examples)

        
        (X_train, Y_train), (X_test, Y_test) = mnist.load_data()
        Y_test = np.squeeze(Y_test)
        X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2],num_channels)
        
        x_test = X_test 
        y_test = Y_test
        
        x_adv = x_test + np.random.uniform(-attack.epsilon, attack.epsilon, x_test.shape)
        

      # per round
        t0 = time.time()
        for epoch in range(int(attack.k/10)):
            np.random.shuffle(idx)
            x_test, x_adv, y_test = x_test[idx], x_adv[idx], y_test[idx]
            permutation = permutation[idx]
            
            for ibatch in range(num_batches):
                bstart = ibatch * eval_batch_size
                bend = min(bstart + eval_batch_size, num_eval_examples)
                
                x_batch = x_test[bstart:bend, :]
                x_batch_adv = x_adv[bstart:bend, :]
                y_batch = y_test[bstart:bend]
            x_adv[bstart:bend, :] = attack.perturb(x_batch, x_batch_adv, y_batch, sess)
            

        inv_permutation = np.argsort(permutation)
        x_adv = x_adv[inv_permutation]
 
        
        t1 = time.time()

        print('restart: ', restart, '   time per batch: ', t1 - t0)

        print('L2: ', np.mean(np.square(x_adv - mnist.test.images)))
        print('Linf: ', np.max(np.abs(x_adv - mnist.test.images)))
        preds = np.argmax(sess.run(attack.model.predict(tf.convert_to_tensor(x_adv))), 1)
        acc = np.mean(np.equal(preds, y_test))
        print('adv acc: ', acc)

c:  1.1
Iterating over 5 batches
0
1
2


In [259]:
# # DAA 4rd try according to tf2 cleverhans 

# # WHY IS X_ADV NEGATIVE?

# from __future__ import absolute_import
# from __future__ import division
# from __future__ import print_function
# from scipy.spatial.distance import pdist, squareform
# from matplotlib import pyplot as plt

# c = 1.1
# import time

# class LinfBLOBAttack:
#     def __init__(self, model, epsilon, k, a, random_start):
#         self.model = model
#         self.epsilon = epsilon
#         self.k = k
#         self.a = a
#         self.rand = random_start

#         c = 1.1
#         print('c: ', c)

#     def perturb(self, x_nat, x_adv, y, sess): 
#         batch_size = x_adv.shape[0]
#         x_adv = tf.convert_to_tensor(x_adv)
#         x_adv = tf.dtypes.cast(x_adv, tf.float32)
#         with tf.GradientTape(persistent=True) as g:
#             g.watch(x_adv)
# #             out = tf.argmax(m4.model(x_nat),axis=1)
# #             out = tf.expand_dims(out, axis=1)
# #             out = tf.dtypes.cast(out, tf.float64)
# #             y = tf.dtypes.cast(y, tf.int64)
# #             loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=out)
#             out = m4.model(x_adv)
#             y = tf.expand_dims(y, axis=1)
#             out = tf.dtypes.cast(out, tf.float32)
#             y = tf.dtypes.cast(y, tf.int64)
#             loss = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=out)
#         print(loss.eval(session=sess))
#         for epoch in range(10):
#             print(epoch)
#             grad = g.gradient(loss, x_adv)
#             grad = grad.eval(session=sess)
#             x_adv = x_adv.eval(session=sess)
#             grad = np.reshape(grad, [grad.shape[0], grad.shape[1]*grad.shape[2]*grad.shape[3]])
#             kxy, dxkxy = self.svgd_kernel(x_adv)
#             x_adv = np.reshape(x_adv, [x_adv.shape[0], x_adv.shape[1]*x_adv.shape[2]*x_adv.shape[3]])
#             x_adv += self.a * np.sign(c*(-(np.matmul(kxy, -grad) + dxkxy)/batch_size) + grad)
            
#             #we need to put back the dimensions since our model has a 4-dimensional input
#             x_adv = np.reshape(x_adv, [x_adv.shape[0], 28, 28, 1])
            
#             x_adv = np.clip(x_adv, x_nat - self.epsilon, x_nat + self.epsilon) 
            
#             x_adv = np.clip(x_adv, 0, 1) # ensure valid pixel range
# #             print(x_adv)
#             x_adv = tf.convert_to_tensor(x_adv)
# #             print(x_adv.eval(session=sess))
#         return x_adv
    
#     def svgd_kernel(self, theta):
#         theta = np.reshape(theta, [theta.shape[0], theta.shape[1]*theta.shape[2]])
#         sq_dist = pdist(theta)
#         pairwise_dists = squareform(sq_dist)**2

#         h = np.median(pairwise_dists)  
#         h = np.sqrt(0.5 * h / np.log(theta.shape[0]))

#         # compute the rbf kernel
#         Kxy = np.exp( -pairwise_dists / h**2 / 2)

#         dxkxy = -np.matmul(Kxy, theta)
#         sumkxy = np.sum(Kxy, axis=1)
#         for i in range(theta.shape[1]):
#             dxkxy[:, i] = dxkxy[:,i] + np.multiply(theta[:,i],sumkxy)
#         dxkxy = dxkxy / (h**2)
#         return (Kxy, dxkxy)


# if __name__ == '__main__':
#     import json
#     import sys
#     import math
    
#     sess =  backend.get_session()

#     attack = LinfBLOBAttack(m4,
#                          epsilon=0.3,
#                          k=200,
#                          a=0.01,
#                          random_start=True)

#     num_eval_examples = 1000
#     eval_batch_size = 200
#     num_batches = int(math.ceil(num_eval_examples / eval_batch_size))

#     print('Iterating over {} batches'.format(num_batches))

#     #     x_adv_final = np.copy(mnist.test.images)

#     for restart in range(50):
#       # Initialize permutation
#         permutation = np.arange(num_eval_examples)
#         idx = np.arange(num_eval_examples)

        
#         (X_train, Y_train), (X_test, Y_test) = mnist.load_data()
#         Y_test = np.squeeze(Y_test)
#         X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2],num_channels)
        
#         x_test = X_test 
#         y_test = Y_test
        
#         x_adv = x_test + np.random.uniform(-attack.epsilon, attack.epsilon, x_test.shape)
        

#       # per round
#         t0 = time.time()
#         for epoch in range(int(attack.k/10)):
#             np.random.shuffle(idx)
#             x_test, x_adv, y_test = x_test[idx], x_adv[idx], y_test[idx]
#             permutation = permutation[idx]
            
#             for ibatch in range(num_batches):
#                 bstart = ibatch * eval_batch_size
#                 bend = min(bstart + eval_batch_size, num_eval_examples)
                
#                 x_batch = x_test[bstart:bend, :]
#                 x_batch_adv = x_adv[bstart:bend, :]
#                 y_batch = y_test[bstart:bend]
#             x_adv[bstart:bend, :] = attack.perturb(x_batch, x_batch_adv, y_batch, sess)
            

#         inv_permutation = np.argsort(permutation)
#         x_adv = x_adv[inv_permutation]
 
        
#         t1 = time.time()

#         print('restart: ', restart, '   time per batch: ', t1 - t0)

#         print('L2: ', np.mean(np.square(x_adv - mnist.test.images)))
#         print('Linf: ', np.max(np.abs(x_adv - mnist.test.images)))
#         preds = np.argmax(sess.run(attack.model.predict(tf.convert_to_tensor(x_adv))), 1)
#         acc = np.mean(np.equal(preds, y_test))
#         print('adv acc: ', acc)

c:  1.1
Iterating over 5 batches
inside


AttributeError: 'numpy.dtype' object has no attribute 'is_floating'

In [4]:
#ODI attack class

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np

class LinfPGDAttack:
    def __init__(self, model, epsilon, num_steps, step_size, random_start, loss_func, batch_size, use_ODI=False):
        self.model = model
        self.epsilon = epsilon
        self.num_steps = num_steps
        self.step_size = step_size
        self.rand = random_start
        self.batch_size = batch_size
        self.use_ODI = use_ODI

        if loss_func == 'xent':
            loss = model.xent
        elif loss_func == 'margin':
            label_mask = tf.one_hot(model.y_input,
                              10,
                              on_value=1.0,
                              off_value=0.0,
                              dtype=tf.float32)
            correct_logit = tf.reduce_sum(label_mask * model.pre_softmax, axis=1)
            wrong_logit = tf.reduce_max((1-label_mask) * model.pre_softmax - 1e4*label_mask, axis=1)
            loss = wrong_logit - correct_logit
      
        self.grad = tf.gradients(loss, model.x_input)[0]

        if self.use_ODI:
            self.rand_direct = tf.Variable(np.zeros((self.batch_size,10)).astype(np.float32),name='rand_direct')
            self.input_placeholder = tf.placeholder(tf.float32, shape=[self.batch_size,10])
            self.assign_op = self.rand_direct.assign(self.input_placeholder)
            loss = tf.tensordot(model.pre_softmax, self.rand_direct,axes=[[0,1],[0,1]])
            self.grad_ODI = tf.gradients(loss, model.x_input)[0]

    def lossSet(self,rand_vector,sess):
        sess.run(self.assign_op,feed_dict={self.input_placeholder: rand_vector.astype(np.float32)})

    def perturb(self, x_org, x_start, y, sess):
        if self.rand:
            x = x_org + np.random.uniform(-self.epsilon, self.epsilon, x_org.shape)
            x = np.clip(x, 0, 255) # ensure valid pixel range
        else:
            x = np.copy(x_start)

        for i in range(self.num_steps):
            if self.use_ODI:
                grad = sess.run(self.grad_ODI, feed_dict={self.model.x_input: x,
                                            self.model.y_input: y})
            else:
                grad = sess.run(self.grad, feed_dict={self.model.x_input: x,
                                            self.model.y_input: y})

            x = np.add(x, self.step_size * np.sign(grad), out=x, casting='unsafe')

            x = np.clip(x, x_org - self.epsilon, x_org + self.epsilon)
            x = np.clip(x, 0, 255) # ensure valid pixel range

        return x


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
import os
import sys
import pickle

import tensorflow as tf
import numpy as np 

import cifar10_input
from model import Model
from pgd_attack import LinfPGDAttack

import argparse

parser = argparse.ArgumentParser(description='CIFAR PGD Attack Evaluation')
parser.add_argument('--save_folder', type=str,default='',
                    help='name of save folder')
parser.add_argument('--data_path', type=str,default='../data/cifar10/',
                    help='path of data folder')
parser.add_argument('--model_path', type=str,default='./models/model_0/checkpoint-70000',
                    help='path of model folder')    
parser.add_argument('--evalSize',
                    default=10000,type=int,
                    help='number of evaluated images')
parser.add_argument('--eval_batch_size',
                    default=100,type=int,
                    help='batch size')
parser.add_argument('--num_restart',
                    default=20,type=int,
                    help='number of restarts')
parser.add_argument('--rand_start',
                    default=1,type=int,
                    help='random initialization for attack')
parser.add_argument('--eps',
                    default=8.0,type=float,
                    help='size of l_inf ball')
parser.add_argument('--step_size',
                    default=2.0,type=float,
                    help='step size for PGD')
parser.add_argument('--num_step',
                    default=20,type=int,
                    help='number of PGD step')
parser.add_argument('--step_size_ODI',
                    default=8.0,type=float,
                    help='step size for ODI')
parser.add_argument('--num_step_ODI',
                    default=2,type=int,
                    help='number of ODI step')                      
args = parser.parse_args()

cifar = cifar10_input.CIFAR10Data(args.data_path)

model = Model(mode='eval')
var_all = tf.get_collection(tf.GraphKeys.VARIABLES)
saver = tf.train.Saver(var_all)

if args.num_step_ODI > 0:
  init_ODI = LinfPGDAttack(model,
                        args.eps,
                        args.num_step_ODI,
                        args.step_size_ODI,
                        args.rand_start,
                        'margin', 
                        args.eval_batch_size,
                        use_ODI=True)
attack_PGD = LinfPGDAttack(model,
                      args.eps,
                      args.num_step,
                      args.step_size,
                      False,
                      'margin',
                      args.eval_batch_size)
          
          
def evaluate_checkpoint(sess):
  # Iterate over the samples batch-by-batch
  num_batches = int(math.ceil(args.evalSize / args.eval_batch_size))
  correct_list = np.ones(args.evalSize)

  for ibatch in range(num_batches):
    bstart = ibatch * args.eval_batch_size
    bend = min(bstart + args.eval_batch_size, args.evalSize)

    x_batch = cifar.eval_data.xs[bstart:bend]
    y_batch = cifar.eval_data.ys[bstart:bend]
    x_batch_org = x_batch
    
    if args.num_step_ODI > 0:
      ran_ = np.random.uniform(-1.0,1.0, (args.eval_batch_size,10))
      init_ODI.lossSet(ran_,sess)
      x_batch = init_ODI.perturb(x_batch_org,x_batch, y_batch, sess)
    elif args.rand_start == 1:
      x_batch = x_batch_org + (np.random.uniform(-args.eps, args.eps, x_batch_org.shape) ) 
      x_batch= np.clip(x_batch, 0., 255.) # ensure valid pixel range

    x_batch = attack_PGD.perturb(x_batch_org, x_batch, y_batch, sess)

    dict_adv = {model.x_input: x_batch,
                model.y_input: y_batch}

    cur_isCorrect, = sess.run([model.correct_prediction],
                                    feed_dict = dict_adv)
    correct_list[bstart:bend] = cur_isCorrect              

  return correct_list
 
#main
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver.restore(sess, args.model_path)

model_dir = 'results/'+args.save_folder
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
is_correct = np.ones(args.evalSize)
acc_curve = np.zeros(args.num_restart)
for i in range(args.num_restart):
    curr_correct = evaluate_checkpoint(sess)
    is_correct = curr_correct * is_correct
    acc_curve[i] = is_correct.mean()
    with open(model_dir+'/result.pk', 'wb') as f:
        pickle.dump([acc_curve,is_correct], f)
print(acc_curve)
        

In [ ]:
# np.save('plots_4/MNIST/tanh_32_diverse_MNIST/tanh_32_diverse_benign.npy', probs_benign)
# np.save('plots_4/MNIST/tanh_32_diverse_MNIST/tanh_32_diverse_pgd.npy', probs_adv_pgd)
# np.save('plots_4/MNIST/tanh_32_diverse_MNIST/tanh_32_diverse_cw.npy', probs_adv_cw)
# np.save('plots_4/MNIST/tanh_32_diverse_MNIST/tanh_32_diverse_bsa.npy', probs_adv_bsa)
# np.save('plots_4/MNIST/tanh_32_diverse_MNIST/tanh_32_diverse_random.npy', probs_random)
# np.save('plots_4/MNIST/tanh_32_diverse_MNIST/tanh_32_diverse_noise.npy', probs_noise)



In [ ]:
# plt.figure(1)
# kernel = stats.gaussian_kde(probs_benign, bw_method=0.5)
# plt.plot(np.arange(0, 1, .01), kernel.pdf(np.arange(0, 1, .01)), linewidth=4)   

# plt.figure(2)
# kernel = stats.gaussian_kde(probs_adv_pgd, bw_method=0.5)
# plt.plot(np.arange(0, 1, .01), kernel.pdf(np.arange(0, 1, .01)), linewidth=4)   

# plt.figure(3)
# kernel = stats.gaussian_kde(probs_adv_cw, bw_method=0.5)
# plt.plot(np.arange(0, 1, .01), kernel.pdf(np.arange(0, 1, .01)), linewidth=4)   

# plt.figure(4)
# kernel = stats.gaussian_kde(probs_adv_bsa, bw_method=0.5)
# plt.plot(np.arange(0, 1, .01), kernel.pdf(np.arange(0, 1, .01)), linewidth=4)   

# plt.figure(5)
# kernel = stats.gaussian_kde(probs_random, bw_method=0.5)
# plt.plot(np.arange(0, 1, .01), kernel.pdf(np.arange(0, 1, .01)), linewidth=4)   

# plt.figure(6)
# kernel = stats.gaussian_kde(probs_noise, bw_method=0.5)
# plt.plot(np.arange(0, 1, .01), kernel.pdf(np.arange(0, 1, .01)), linewidth=4)   

In [ ]:
# m4.X_test, m4.Y_test y_adv_pgd, X_adv_pgd

In [ ]:
#for U=2 we chose randomly the lwta__dense__activation_32 for LogEnsemble for CIFAR10
#for U=4 we chose lwta__dense__activation_2 for Tanh16 for CIFAR10

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
import itertools
import numpy as np
import matplotlib.pyplot as plt

def transform_activations():
    acts_l1 = []
    
    for i in range(10):
        indices = np.where(y_adv_pgd==i)
        adv_input = X_adv_pgd[indices]
        probabilities = m4.model_full.get_layer('lwta__dense__activation_5').output[1]
        intermediate_layer_model = Model(inputs=m4.model_full.input,outputs=probabilities)
        acts_1 = intermediate_layer_model.predict(adv_input) #[1,:,:]

        acts_1_mean = acts_1.mean(0)
        acts_1 = np.reshape(acts_1_mean, [-1])
        acts_l1.append(acts_1)
        
    return np.array(acts_l1)

activations = transform_activations() #change C and U according to the model


In [ ]:
#for 2 Units 

def plot_activations(activations, C, U):
    
    if not os.path.exists('svg'):
        os.mkdir('svg')
        
    fig = plt.figure(figsize=(20,10))
    acts = activations[:,:16]
#     acts = activations
#     acts = np.expand_dims(acts, axis=1)
    ix = plt.imshow(acts, aspect='equal', cmap='binary')
    ax = plt.gca()
    size = acts.shape[-1]
    
    ax.set_xticks(np.arange(-.5, size, U))
    ax.set_yticks(np.arange(0,10,1))

    ax.set_xticklabels(np.arange(0,size+1, U))
    ax.set_yticklabels(np.arange(0,10,1))

    ax.set_xticks(np.arange(.51, size, 1), minor=True)
    ax.set_yticks(np.arange(.51, 10, 1), minor=True);
    ax.tick_params(labelsize=14)

    ax.grid(which='minor', color=(240/255.,240./255,240/255.), linestyle=':', linewidth=1)
    ax.xaxis.grid(True,'major', color='black', linewidth=2)
    ax.yaxis.grid(True,'minor')
    
    ax.annotate('Block 1', xy=(0.07, 1.04), xytext=(0.07, 1.06), xycoords='axes fraction', 
            fontsize=15, ha='center', va='bottom',
            bbox=dict(boxstyle='square', fc='white'),
            arrowprops=dict(arrowstyle='-[, widthB=2, lengthB=1.5', lw=2.0))

    ax.annotate('Block 2', xy=(0.19, 1.04), xytext=(0.19, 1.06), xycoords='axes fraction', 
            fontsize=15, ha='center', va='bottom',
            bbox=dict(boxstyle='square', fc='white'),
            arrowprops=dict(arrowstyle='-[, widthB=2, lengthB=1.5', lw=2.0))
    
    ax.annotate('Block 3', xy=(0.31, 1.04), xytext=(0.31, 1.06), xycoords='axes fraction', 
        fontsize=15, ha='center', va='bottom',
        bbox=dict(boxstyle='square', fc='white'),
        arrowprops=dict(arrowstyle='-[, widthB=2, lengthB=1.5', lw=2.0))
    
    ax.annotate('Block 4', xy=(0.43, 1.04), xytext=(0.43, 1.06), xycoords='axes fraction', 
        fontsize=15, ha='center', va='bottom',
        bbox=dict(boxstyle='square', fc='white'),
        arrowprops=dict(arrowstyle='-[, widthB=2, lengthB=1.5', lw=2.0))
    
    ax.annotate('Block 5', xy=(0.56, 1.04), xytext=(0.56, 1.06), xycoords='axes fraction', 
        fontsize=15, ha='center', va='bottom',
        bbox=dict(boxstyle='square', fc='white'),
        arrowprops=dict(arrowstyle='-[, widthB=2, lengthB=1.5', lw=2.0))
    
    ax.annotate('Block 6', xy=(0.69, 1.04), xytext=(0.69, 1.06), xycoords='axes fraction', 
        fontsize=15, ha='center', va='bottom',
        bbox=dict(boxstyle='square', fc='white'),
        arrowprops=dict(arrowstyle='-[, widthB=2, lengthB=1.5', lw=2.0))
    
    ax.annotate('Block 7', xy=(0.82, 1.04), xytext=(0.82, 1.06), xycoords='axes fraction', 
        fontsize=15, ha='center', va='bottom',
        bbox=dict(boxstyle='square', fc='white'),
        arrowprops=dict(arrowstyle='-[, widthB=2, lengthB=1.5', lw=2.0))
    
    ax.annotate('Block 8', xy=(0.94, 1.04), xytext=(0.94, 1.06), xycoords='axes fraction', 
        fontsize=15, ha='center', va='bottom',
        bbox=dict(boxstyle='square', fc='white'),
        arrowprops=dict(arrowstyle='-[, widthB=2, lengthB=1.5', lw=2.0))

    plt.xlabel('Units',fontsize=20)
    plt.ylabel('Classes',fontsize=20)
    
    plt.colorbar(fraction=0.025, pad=0.01)
    plt.clim(0, 1);
 
#     plt.show()
    plt.savefig('svg/cifar_pgd_softmax_U_2_nineth.eps', format='eps',bbox_inches='tight')
#         tikz_save(path+'tikz\\layer_'+str(i)+'all_digits.tex',  figureheight='20cm', figurewidth='20cm')
#     plt.close(fig)

# activations_final = []
# for i in range(10):
#     activations_final.append(activations[i][:20])
# activations_final = np.asarray(activations_final)
plot_activations(activations=activations, C=10, U=2)

In [ ]:
# #for 4 Units

# def plot_activations(activations, C, U):
    
#     combs = []
#     for i in range(C):
#         for j in range(i, C):
#             combs.append((i,j))
            
#     a = list(range(C))
#     b = list(range(C))
#     combs_2 = list(itertools.product(a, b))
    
#     if not os.path.exists('svg'):
#         os.mkdir('svg')
        
#     fig = plt.figure(figsize=(20,10))
#     acts = activations[:,:16]
# #     acts = np.expand_dims(acts, axis=1)
#     ix = plt.imshow(acts, aspect='equal', cmap='binary')
#     ax = plt.gca()
#     size = acts.shape[-1]
    
#     ax.set_xticks(np.arange(-.5, size, U))
#     ax.set_yticks(np.arange(0,10,1))

#     ax.set_xticklabels(np.arange(0,size+1, U))
#     ax.set_yticklabels(np.arange(0,10,1))

#     ax.set_xticks(np.arange(.51, size, 1), minor=True)
#     ax.set_yticks(np.arange(.51, 10, 1), minor=True);
#     ax.tick_params(labelsize=14)


#     ax.grid(which='minor', color=(240/255.,240./255,240/255.), linestyle=':', linewidth=1)
#     ax.xaxis.grid(True,'major', color='black', linewidth=2)
#     ax.yaxis.grid(True,'minor')
    
#     ax.annotate('Block 1', xy=(0.13, 1.025), xytext=(0.13, 1.07), xycoords='axes fraction', 
#             fontsize=10, ha='center', va='bottom',
#             bbox=dict(boxstyle='square', fc='white'),
#             arrowprops=dict(arrowstyle='-[, widthB=8, lengthB=1.5', lw=2.0))
    
#     ax.annotate('Block 2', xy=(0.38, 1.025), xytext=(0.38, 1.07), xycoords='axes fraction', 
#             fontsize=10, ha='center', va='bottom',
#             bbox=dict(boxstyle='square', fc='white'),
#             arrowprops=dict(arrowstyle='-[, widthB=8, lengthB=1.5', lw=2.0))
    
#     ax.annotate('Block 3', xy=(0.63, 1.025), xytext=(0.63, 1.07), xycoords='axes fraction', 
#             fontsize=10, ha='center', va='bottom',
#             bbox=dict(boxstyle='square', fc='white'),
#             arrowprops=dict(arrowstyle='-[, widthB=8, lengthB=1.5', lw=2.0))
    
#     ax.annotate('Block 4', xy=(0.88, 1.025), xytext=(0.88, 1.07), xycoords='axes fraction', 
#             fontsize=10, ha='center', va='bottom',
#             bbox=dict(boxstyle='square', fc='white'),
#             arrowprops=dict(arrowstyle='-[, widthB=8, lengthB=1.5', lw=2.0))

    
#     plt.xlabel('Units',fontsize=20)
#     plt.ylabel('Classes',fontsize=20)
    
#     plt.colorbar(fraction=0.0243, pad=0.01)
#     plt.clim(0, 1);
 
# #     plt.show()
#     plt.savefig('svg/cifar_pgd_tanh16_U_4.eps', format='eps',bbox_inches='tight')
#         #tikz_save(path+'tikz\\layer_'+str(i)+'all_digits.tex',  figureheight='20cm', figurewidth='20cm')
# #     plt.close(fig)

# plot_activations(activations=activations, C=16, U=4)